In [10]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet("../serverless.parquet")

# View all columns and their data types
print(df.dtypes)

KeyboardInterrupt: 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, hour, avg, min, max, lag, count
from pyspark.sql.window import Window

# Create Spark session
spark = SparkSession.builder \
    .appName("ParquetFileExplorer") \
    .getOrCreate()

In [2]:
# Load the Parquet file
df = spark.read.parquet("../provisioned.parquet")

In [3]:
# df.show(20)
from pyspark.sql.functions import col, to_date

# df.show(200)
# df.filter(df["cache_source_query_id"].isNotNull()).show(20)
# df.filter(df["cluster_size"].isNotNull()).show(20)
# df.filter((df["cache_source_query_id"].isNotNull()) & (df["feature_fingerprint"].isNotNull())).show(20)
df.filter((df["was_cached"].isNotNull()) & (df["feature_fingerprint"].isNotNull())).show(20)


# whole_count = df.count()
# print(whole_count)
# one_day_count = df.filter(to_date(col("arrival_timestamp")) == "2024-03-01").count()
# print(one_day_count)
# print("One day makes up n %% of total days:", (1/90)*100)
# print("One day's logs make up n %% of all logs:", (one_day_count/whole_count)*100)

+-----------+------------+-------+-----------+--------+--------------------+-------------------+-----------------+---------------------+--------------------+-----------+----------+---------------------+----------+-----------------------------+----------------------------+--------------------------+--------------------+---------------+--------------+--------------+---------+---------+----------------+
|instance_id|cluster_size|user_id|database_id|query_id|   arrival_timestamp|compile_duration_ms|queue_duration_ms|execution_duration_ms| feature_fingerprint|was_aborted|was_cached|cache_source_query_id|query_type|num_permanent_tables_accessed|num_external_tables_accessed|num_system_tables_accessed|      read_table_ids|write_table_ids|mbytes_scanned|mbytes_spilled|num_joins|num_scans|num_aggregations|
+-----------+------------+-------+-----------+--------+--------------------+-------------------+-----------------+---------------------+--------------------+-----------+----------+------------

In [ ]:
from pyspark.sql import SparkSession
from confluent_kafka import Producer
from confluent_kafka.admin import (AdminClient, NewTopic, ConfigResource)
# from config import config
import json

from datetime import datetime
import time

# return True if topic exists and False if not
def topic_exists(admin, topic):
    print("checking if topic exists...")
    metadata = admin.list_topics()
    for t in iter(metadata.topics.values()):
        print("Topic:", t.topic)
        if t.topic == topic:
            return True
    return False


# Function to convert row to JSON serializable format
def row_to_dict(row):
    row_dict = row.asDict()
    for key, value in row_dict.items():
        if isinstance(value, datetime):
            row_dict[key] = value.isoformat()
    return row_dict

# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaPublisher") \
    .getOrCreate()

# Load your dataset into a Spark DataFrame
df = spark.read.parquet("../serverless.parquet")

# Initialize Kafka producer with correct configuration
producer = Producer({
    'bootstrap.servers': 'localhost:9092',  # Correct configuration key
    'group.id': 'redset_stream',
})

# admin = AdminClient({
#     'bootstrap.servers': 'localhost:9092',  # Correct configuration key
#     'group.id': 'assignment5teachers',
# })
# if topic_exists(admin, "sagnik"):
#     print("TOPIC EXISTS!")
# else:
#     print("topic is fked")

# Continuously stream data

admin = AdminClient({'bootstrap.servers': 'localhost:9092'})

# Create topic if it doesn't exist
# if not topic_exists(admin, kafka_topic):
#     print(f"Creating topic: {kafka_topic}")
#     new_topic = NewTopic(kafka_topic, num_partitions=1, replication_factor=1)
#     admin.create_topics([new_topic])

while True:
    # Select the first 20 rows
    first_20_rows = df.limit(20).collect()
    msg = producer.poll(3.0)  # Increased polling timeout for better chance of receiving messages

    # Publish each row to Kafka
    for row in first_20_rows:
        message = row_to_dict(row)
        print(f"Broadcasting message: {message}")
        producer.produce('muhid', value=json.dumps(message))  # Sending serialized JSON
        producer.poll(0)  # Poll for background events to ensure messages are sent
        # producer.flush()
    # Sleep for a while before sending the next batch
    time.sleep(5)

# Close the producer (this will never be reached in this infinite loop)
producer.flush()
producer.close()

In [ ]:
from pyspark.sql import SparkSession
from confluent_kafka import Producer
from confluent_kafka.admin import (AdminClient, NewTopic, ConfigResource)
# from config import config
import json

from datetime import datetime
import time


# Function to convert row to JSON serializable format
def row_to_dict(row):
    row_dict = row.asDict()
    for key, value in row_dict.items():
        if isinstance(value, datetime):
            row_dict[key] = value.isoformat()
    return row_dict

# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaPublisher") \
    .getOrCreate()

# Load your dataset into a Spark DataFrame
df = spark.read.parquet("../serverless.parquet")

# Initialize Kafka producer with correct configuration
producer = Producer({
    'bootstrap.servers': 'localhost:9092',  # Correct configuration key
    'group.id': 'redset_stream',
})

while True:
    # Select the first 20 rows
    first_20_rows = df.limit(20).collect()
    msg = producer.poll(3.0)  # Increased polling timeout for better chance of receiving messages

    # Publish each row to Kafka
    for row in first_20_rows:
        message = row_to_dict(row)
        print(f"Broadcasting message: {message}")
        producer.produce('muhid', value=json.dumps(message))  # Sending serialized JSON
        producer.poll(0)  # Poll for background events to ensure messages are sent
        # producer.flush()
    # Sleep for a while before sending the next batch
    time.sleep(5)

# Close the producer (this will never be reached in this infinite loop)
producer.flush()
producer.close()

In [ ]:
from pyspark.sql import SparkSession
from confluent_kafka import Producer
import json
import time
from datetime import datetime

# Function to convert row to JSON serializable format
def row_to_dict(row):
    row_dict = row.asDict()
    for key, value in row_dict.items():
        if isinstance(value, datetime):
            row_dict[key] = value.isoformat()
    return row_dict

# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaPublisher") \
    .getOrCreate()

# Load your dataset into a Spark DataFrame
df = spark.read.parquet("../serverless.parquet")

# Initialize Kafka producer with optimized configuration
producer = Producer({
    'bootstrap.servers': 'localhost:9092',  # Correct configuration key
    'acks': '1',  # Acknowledgment from the leader broker
    'batch.size': 16384,  # Batch size in bytes (16 KB)
    'linger.ms': 5,  # Time to wait before sending a batch (in ms)
    'max.in.flight.requests.per.connection': 5,  # Number of unacknowledged requests to allow
    'compression.type': 'snappy',  # Use compression to reduce payload size
    'group.id': 'redset_stream',
})

# Convert DataFrame to RDD and use zipWithIndex to add row indices
rdd = df.rdd.zipWithIndex()

# Batch size
batch_size = 1
index = 0

while True:
    # Get a batch of rows (index, row) from the RDD
    batch = rdd.filter(lambda x: x[1] >= index and x[1] < index + batch_size).map(lambda x: x[0]).collect()
    
    # If there are no more rows, break the loop
    if not batch:
        break
    
    # Prepare batch messages
    messages = [json.dumps(row_to_dict(row)) for row in batch]
    
    # Send all messages in one go to Kafka
    for message in messages:
        producer.produce('muhid', value=message)  # Sending serialized JSON

    # Poll and flush producer to ensure delivery (done less frequently)
    producer.poll(1.0)  # Polling every 1 second to ensure background events are handled
    producer.flush()  # Flush after sending the batch

    # Increment index for the next batch
    index += batch_size

    # Sleep for a while before sending the next batch
    time.sleep(2)  # Adjust sleep time as needed to balance performance and system load

# Close the producer
producer.flush()
producer.close()


In [ ]:
from pyspark.sql import SparkSession
from confluent_kafka import Producer
import json
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# Function to convert row to JSON serializable format
def row_to_dict(row):
    row_dict = row.asDict()
    for key, value in row_dict.items():
        if isinstance(value, datetime):
            row_dict[key] = value.isoformat()
    return row_dict

# Function to send a single message to Kafka
def send_to_kafka(message, producer):
    try:
        producer.produce('muhid', value=message)  # Sending serialized JSON
        producer.poll(0)  # Poll for background events to ensure messages are sent
    except Exception as e:
        print(f"Error sending message: {e}")

print("1. Creating spark session...")
# Create Spark session
spark = SparkSession.builder \
    .appName("KafkaPublisher") \
    .getOrCreate()

print("2. Reading parquet file...")
# Load your dataset into a Spark DataFrame
df = spark.read.parquet("../serverless.parquet")

# Initialize Kafka producer with optimized configuration
producer = Producer({
    'bootstrap.servers': 'localhost:9092',  # Correct configuration key
    'acks': '1',  # Acknowledgment from the leader broker
    'batch.size': 16384,  # Batch size in bytes (16 KB)
    'linger.ms': 5,  # Time to wait before sending a batch (in ms)
    'max.in.flight.requests.per.connection': 5,  # Number of unacknowledged requests to allow
    'compression.type': 'snappy',  # Use compression to reduce payload size
    'group.id': 'redset_stream',
})

print("3. Converting df to rdd...")
# Convert DataFrame to RDD and use zipWithIndex to add row indices
rdd = df.rdd.zipWithIndex()

# Initialize index
index = 0

# Use ThreadPoolExecutor for parallelizing the sending process
with ThreadPoolExecutor(max_workers=5) as executor:
    print("4. Entered thread...")
    while True:
        print("5. Getting one row from RDD...")
        # Get a single row from the RDD (indexed by 'index')
        row = rdd.filter(lambda x: x[1] == index).map(lambda x: x[0]).collect()

        # If the row exists, process and send to Kafka
        if row:
            message = json.dumps(row_to_dict(row[0]))
            print(f"Broadcasting message: {message}")

            # Submit the message sending to Kafka in parallel
            print("6. Sending to kafka...")
            executor.submit(send_to_kafka, message, producer)
            # Increment index for the next row
            index += 1
        else:
            # If no row is found, break the loop (end of DataFrame)
            break

        # Sleep for a while before sending the next row
        time.sleep(0.1)  # You can adjust this based on throughput needs

# Close the producer
producer.flush()
producer.close()


In [ ]:
from pyspark.sql import SparkSession
from confluent_kafka import Producer
from pyspark.sql.functions import col, unix_timestamp, window
import time
import json
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# Initialize Kafka producer with optimized configuration
producer = Producer({
    'bootstrap.servers': 'localhost:9092',  # Correct configuration key
    'acks': '1',  # Acknowledgment from the leader broker
    'batch.size': 500000,  # Batch size in bytes (16 KB)
    'linger.ms': 5,  # Time to wait before sending a batch (in ms)
    'max.in.flight.requests.per.connection': 5,  # Number of unacknowledged requests to allow
    'compression.type': 'snappy',  # Use compression to reduce payload size
    'group.id': 'redset_stream',
})

def stream_data_in_batches(file_path, time_interval_min):
    # Initialize a Spark session
    spark = SparkSession.builder \
        .appName("Data Streaming") \
        .getOrCreate()

    # Read the Parquet file
    df = spark.read.parquet(file_path)

    # Convert arrival_timestamp to timestamp type
    df = df.withColumn("arrival_timestamp", col("arrival_timestamp").cast("timestamp"))

    # Calculate the total duration of the data in seconds
    min_timestamp = df.agg({"arrival_timestamp": "min"}).collect()[0][0]
    max_timestamp = df.agg({"arrival_timestamp": "max"}).collect()[0][0]
    total_duration_sec = (max_timestamp - min_timestamp).total_seconds()

    # Calculate the number of records to send per second
    total_records = df.count()
    records_per_sec = total_records / (time_interval_min * 60)

    # Calculate the batch size (number of records to send per second)
    batch_size = int(records_per_sec)

    # Sort the DataFrame by arrival_timestamp
    df = df.orderBy("arrival_timestamp")

    # Stream the data in batches
    start_time = time.time()
    for i in range(0, total_records, batch_size):
        batch_df = df.limit(batch_size).collect()
        batch_count = len(batch_df)
        print(f"Sending batch {i // batch_size + 1}: {batch_count}")
        print(f"Remaining batches {total_records // batch_size + 1 - i // batch_size + 1}")
        receiving_batch(batch_df, i // batch_size + 1)
        # Sleep
        time.sleep(1)

    # Calculate the total streaming time
    end_time = time.time()
    streaming_duration = end_time - start_time
    print(f"Total streaming duration: {streaming_duration} seconds")


def receiving_batch(batch_df, batch_no):

    # Prepare batch messages
    # messages = [json.dumps(row_to_dict(row)) for row in batch_df]
    messages = json.dumps(batch_df, indent=4, sort_keys=True, default=str)
    try:
        # producer.produce('muhid', value="hello dear")  # Sending serialized JSON
        producer.produce('muhid', value=messages)  # Sending serialized JSON
        print("Sent batch: ", len(batch_df), " Batch No: ", batch_no)
    except Exception as e:
        print(f"Error producing: {e}")

    # Send all messages in one go to Kafka
    # for message in messages:
    #     producer.produce('muhid', value=message)  # Sending serialized JSON

    # Poll and flush producer to ensure delivery (done less frequently)
    producer.poll(1.0)  # Polling every 1 second to ensure background events are handled
    producer.flush()  # Flush after sending the batch
    

# Give the FIle Path and total time you plan to get the entire dataset
stream_data_in_batches('../serverless.parquet', 20000)

### RDD ATTEMPT FOR FASTER ROW ITERATIONS

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from confluent_kafka import Producer
import threading
import json
import time
import math

print("Creating spark session...")
# ✅ Initialize Spark
# Set max partition size to 500MB
spark = SparkSession.builder \
    .appName("ParquetToKafka") \
    .config("spark.sql.files.maxPartitionBytes", str(500 * 1024 * 1024)) \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()


# ✅ Kafka Configuration
KAFKA_CONFIG = {
    "bootstrap.servers": "localhost:9092",  # Change as needed
    "queue.buffering.max.messages": 100000,  # Larger buffer
    "batch.num.messages": 10000,  # Send messages in batches
    "linger.ms": 500  # Delay to allow batch sending
}
TOPIC = "muhid"

print("Reading parquet...")
# ✅ Read Parquet File (Partitioned Automatically into ~500MB chunks)
FILE_PATH = "../provisioned.parquet"
df = spark.read.parquet(FILE_PATH)

print("Converting timestamp to str...")
# ✅ Convert all Timestamp columns to String (Kafka requires JSON serializable data)
for field in df.schema.fields:
    if "timestamp" in str(field.dataType).lower():
        df = df.withColumn(field.name, col(field.name).cast("string"))


# Get estimated size in bytes
# size_in_bytes = spark._jsparkSession.sessionState().executePlan(df._jdf.queryExecution().logical()).optimizedPlan().stats().sizeInBytes()

# Calculate the number of partitions (~500MB each)
# num_partitions = max(1, math.ceil(size_in_bytes / (500 * 1024 * 1024)))

print("Repartitioning...")
# ✅ Repartition the DataFrame
df = df.repartition(34)


# ✅ Kafka Producer Function
def kafka_producer():
    return Producer(KAFKA_CONFIG)

# ✅ Function to Process Each Partition
def process_partition(partition):
    producer = kafka_producer()
    message_count = 0

    for row in partition:
        message = json.dumps(row.asDict())  # Convert Spark Row to JSON
        
        while True:  # Handle queue full issue
            try:
                print("Sending to kafka...")
                producer.produce(TOPIC, key=str(row[0]), value=message)
                message_count += 1
                break  # Exit retry loop if successful
            except BufferError:
                print("⚠️ Kafka queue full. Flushing and retrying...")
                producer.flush()
                time.sleep(1)  # Small delay before retrying
        
        if message_count % 10000 == 0:
            producer.flush()

    producer.flush()
    print(f"✅ Partition Processed: {threading.current_thread().name}")

print("Running threads...")
# ✅ Run Threads for Each Partition
# threads = []
# thread_count = 0
# for partition in df.rdd.glom().collect():  # Collect partitions as list of rows
#     print(f"Thread {thread_count} created")
#     thread = threading.Thread(target=process_partition, args=(partition,))
#     threads.append(thread)
#     thread.start()
#     thread_count += 1

threads = []
thread_count = 0  # Use a global counter

def process_partition_wrapper(partition):
    global thread_count
    print(f"Thread {thread_count} created")
    
    thread = threading.Thread(target=process_partition, args=(partition,))
    threads.append(thread)
    thread.start()
    
    thread_count += 1  # Increment after starting the thread

df.rdd.foreachPartition(process_partition_wrapper)

print("Waiting for threads to finish...")
# ✅ Wait for all threads to finish
for thread in threads:
    thread.join()

print("✅ Done! Parquet file streamed to Kafka.")

# ✅ Stop Spark Session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
import os

# ✅ Initialize Spark Session
spark = SparkSession.builder \
    .appName("ParquetSplitter") \
    .config("spark.driver.memory", "17g") \
    .config("spark.executor.memory", "17g") \
    .config("spark.sql.shuffle.partitions", "34") \
    .getOrCreate()


# ✅ Configuration: Set Input & Output Paths + Number of Splits
input_parquet = "../provisioned.parquet"  # Replace with actual path
output_folder = "../provisioned_splits"       # Folder where splits will be stored
num_splits = 34                               # Change this to control the number of partitions

# ✅ Ensure Output Directory Exists
os.makedirs(output_folder, exist_ok=True)

# ✅ Read Parquet File
df = spark.read.parquet(input_parquet)

# ✅ Repartition Dataframe into `num_splits`
df = df.repartition(num_splits)
# df = df.repartition(int(df.count() / (500 * 1024 * 1024)))

# ✅ Save Each Partition as a Separate Parquet File
df.write.mode("overwrite").parquet(output_folder)

# ✅ Print Success Message
print(f"Parquet file split into {num_splits} parts and saved in '{output_folder}'")

# ✅ Stop Spark Session
spark.stop()

Using pyarrow to stream batches

In [2]:
import pyarrow.parquet as pq
import confluent_kafka
import json
from datetime import datetime
import time

# Custom JSON encoder to handle Timestamp
def json_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')

def stream_parquet_to_kafka(parquet_file_path, kafka_topic, bootstrap_servers):
    # Start total time tracking
    total_start_time = time.time()
    print(f"Starting to stream Parquet file: {parquet_file_path}")

    # Configure Kafka producer
    producer_config = {
        'bootstrap.servers': bootstrap_servers,
        'client.id': 'parquet-streamer'
    }
    producer = confluent_kafka.Producer(producer_config)

    # Open Parquet file and get file metadata
    parquet_file = pq.ParquetFile(parquet_file_path)
    total_rows = parquet_file.metadata.num_rows
    print(f"Total rows in Parquet file: {total_rows}")

    # Tracking variables
    total_rows_processed = 0
    total_batch_time = 0
    total_kafka_time = 0
    # file_writer = 0

    # Iterate through batches instead of individual rows
    for batch_num, batch in enumerate(parquet_file.iter_batches(batch_size=1000), 1):
        batch_start_time = time.time()
        
        # Convert batch to pandas for easier processing
        df_batch = batch.to_pandas()
        batch_conversion_time = time.time() - batch_start_time
        print(f"Batch {batch_num}: Pandas conversion time = {batch_conversion_time:.2f} seconds")

        # Convert entire batch to records and stream
        kafka_start_time = time.time()
        # Convert entire batch to JSON and stream as a single message
        batch_messages = json.dumps(df_batch.to_dict('records'), default=json_serializer)
        producer.produce(
            topic=kafka_topic, 
            value=batch_messages.encode('utf-8')
        )

        # # (don't remove) used to save and monitor batches
        # if file_writer < 10:
        #     f = open(f"../test_provisioned_batches/{file_writer}.txt", "w")
        #     f.write(batch_messages)
        #     f.close()
        #     file_writer += 1

        # Flush to ensure messages are sent
        producer.flush()
        
        kafka_time = time.time() - kafka_start_time
        total_kafka_time += kafka_time

        total_rows_processed += len(df_batch)
        batch_total_time = time.time() - batch_start_time
        total_batch_time += batch_total_time

        print(f"Batch {batch_num}: Kafka production time = {kafka_time:.2f} seconds")
        print(f"Batch {batch_num}: Total batch processing time = {batch_total_time:.2f} seconds")
        print(f"Rows processed so far: {total_rows_processed} / {total_rows}")

    # Close producer
    producer.close()

    # Print final statistics
    total_runtime = time.time() - total_start_time
    print("\n--- Streaming Complete ---")
    print(f"Total runtime: {total_runtime:.2f} seconds")
    print(f"Total rows processed: {total_rows_processed}")
    print(f"Total Kafka production time: {total_kafka_time:.2f} seconds")
    print(f"Average batch processing time: {total_batch_time / batch_num:.2f} seconds")

# Example usage
stream_parquet_to_kafka(
    parquet_file_path='../provisioned.parquet', 
    kafka_topic='muhid', 
    bootstrap_servers='localhost:9092'
)

Starting to stream Parquet file: ../provisioned.parquet
Total rows in Parquet file: 433107811
Batch 1: Pandas conversion time = 0.00 seconds
Batch 1: Kafka production time = 0.96 seconds
Batch 1: Total batch processing time = 0.96 seconds
Rows processed so far: 1000 / 433107811
Batch 2: Pandas conversion time = 0.00 seconds
Batch 2: Kafka production time = 0.06 seconds
Batch 2: Total batch processing time = 0.06 seconds
Rows processed so far: 2000 / 433107811
Batch 3: Pandas conversion time = 0.00 seconds
Batch 3: Kafka production time = 0.06 seconds
Batch 3: Total batch processing time = 0.06 seconds
Rows processed so far: 3000 / 433107811
Batch 4: Pandas conversion time = 0.00 seconds
Batch 4: Kafka production time = 0.08 seconds
Batch 4: Total batch processing time = 0.08 seconds
Rows processed so far: 4000 / 433107811
Batch 5: Pandas conversion time = 0.00 seconds
Batch 5: Kafka production time = 0.06 seconds
Batch 5: Total batch processing time = 0.06 seconds
Rows processed so far

KeyboardInterrupt: 